<a href="https://colab.research.google.com/github/Jum-eng/Social-Media-Mining-Final_Project/blob/main/Multimodal_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import numpy as np
import pandas as pd
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Input, Dense, LSTM, Flatten, Concatenate, GlobalAveragePooling2D, Attention
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score


In [23]:
import pandas as pd
from transformers import BertTokenizer

# Load Text Data
text_data = pd.read_csv('/content/Mental-Health-Twitter.csv')

# Initialize the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Preprocessing Text Data
def preprocess_text(text):
    return tokenizer.encode(text, add_special_tokens=True, truncation=True, padding='max_length', max_length=128)

# Apply preprocessing to create 'input_ids'
text_data['input_ids'] = text_data['post_text'].apply(preprocess_text)

# Verify the columns
print("Text Data Columns After Preprocessing:", text_data.columns)
print(text_data.head())


Text Data Columns After Preprocessing: Index(['Unnamed: 0', 'post_id', 'post_created', 'post_text', 'user_id',
       'followers', 'friends', 'favourites', 'statuses', 'retweets', 'label',
       'input_ids'],
      dtype='object')
   Unnamed: 0             post_id                    post_created  \
0           0  637894677824413696  Sun Aug 30 07:48:37 +0000 2015   
1           1  637890384576778240  Sun Aug 30 07:31:33 +0000 2015   
2           2  637749345908051968  Sat Aug 29 22:11:07 +0000 2015   
3           3  637696421077123073  Sat Aug 29 18:40:49 +0000 2015   
4           4  637696327485366272  Sat Aug 29 18:40:26 +0000 2015   

                                           post_text     user_id  followers  \
0  It's just over 2 years since I was diagnosed w...  1013187241         84   
1  It's Sunday, I need a break, so I'm planning t...  1013187241         84   
2  Awake but tired. I need to sleep but my brain ...  1013187241         84   
3  RT @SewHQ: #Retro bears make perfe

In [24]:
# Load Text Data
text_data = pd.read_csv('/content/Mental-Health-Twitter.csv')
temporal_data = pd.read_csv('/content/city_temperature.csv')
image_data = pd.read_csv('/content/fer2013.csv')

# Preprocessing Text Data
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
text_data['input_ids'] = text_data['post_text'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True, truncation=True, padding='max_length', max_length=128))

# Correct temporal features column names
temporal_features = ['Year', 'Month', 'Day', 'AvgTemperature']

# Standardize the temporal data
scaler = StandardScaler()
temporal_data[temporal_features] = scaler.fit_transform(temporal_data[temporal_features])

# Check the size of each entry in the 'pixels' column
image_data['pixel_size'] = image_data['pixels'].apply(lambda x: len(x.split()))
print(image_data['pixel_size'].value_counts())


# Filter out rows where the pixel size is not 2304
image_data = image_data[image_data['pixel_size'] == 2304]

# Now apply the reshaping process again
image_data['pixels'] = image_data['pixels'].apply(lambda x: np.array(x.split(), dtype='float32').reshape(48, 48, 1))

# Resize to 256x256 and convert to 3 channels
image_data['pixels'] = image_data['pixels'].apply(lambda x: np.resize(x, (256, 256, 3)))



pixel_size
2304    5123
420        1
Name: count, dtype: int64


In [25]:
# Function to process pixel data
def process_pixels(pixel_string):
    try:
        # Convert pixel string to a numpy array of floats
        pixel_array = np.array(pixel_string.split(), dtype='float32')

        # Check if the number of pixels matches the expected size (48x48 for grayscale)
        if pixel_array.size == 48*48:
            # Reshape to (48, 48, 1) for grayscale images
            resized_image = pixel_array.reshape((48, 48, 1))
        elif pixel_array.size == 48*48*3:
            # Reshape to (48, 48, 3) for RGB images
            resized_image = pixel_array.reshape((48, 48, 3))
        else:
            # If the size doesn't match expected dimensions, return an empty image or handle error
            resized_image = np.zeros((48, 48, 1))
    except Exception as e:
        print(f"Error processing pixels: {e}")
        resized_image = np.zeros((48, 48, 1))  # Default value in case of error

    return resized_image

# Apply the processing function to the 'pixels' column
image_data['pixels'] = image_data['pixels'].apply(process_pixels)

# Display the DataFrame to verify changes
print(image_data.head())


Streaming output truncated to the last 5000 lines.
Error processing pixels: 'numpy.ndarray' object has no attribute 'split'
Error processing pixels: 'numpy.ndarray' object has no attribute 'split'
Error processing pixels: 'numpy.ndarray' object has no attribute 'split'
Error processing pixels: 'numpy.ndarray' object has no attribute 'split'
Error processing pixels: 'numpy.ndarray' object has no attribute 'split'
Error processing pixels: 'numpy.ndarray' object has no attribute 'split'
Error processing pixels: 'numpy.ndarray' object has no attribute 'split'
Error processing pixels: 'numpy.ndarray' object has no attribute 'split'
Error processing pixels: 'numpy.ndarray' object has no attribute 'split'
Error processing pixels: 'numpy.ndarray' object has no attribute 'split'
Error processing pixels: 'numpy.ndarray' object has no attribute 'split'
Error processing pixels: 'numpy.ndarray' object has no attribute 'split'
Error processing pixels: 'numpy.ndarray' object has no attribute 'split'


In [26]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Load the dataset
temperature_data = pd.read_csv('/content/city_temperature.csv')

# Inspect the first few rows to understand its structure
print(temperature_data.head())

# Function to preprocess temporal data
def preprocess_temporal(data):
    try:
        # Select relevant columns
        data = data[['region', 'country', 'state', 'city', 'year', 'month', 'day', 'average_temperature']]

        # Handle missing values
        data.fillna(method='ffill', inplace=True)

        # Convert categorical features to numerical using one-hot encoding
        data = pd.get_dummies(data, columns=['region', 'country', 'state', 'city'])

        # Normalize numerical features
        scaler = StandardScaler()
        data[['average_temperature']] = scaler.fit_transform(data[['average_temperature']])

    except Exception as e:
        print(f"Error preprocessing temporal data: {e}")
        return pd.DataFrame()  # Return an empty DataFrame in case of error

    return data

# Apply preprocessing function to the dataset
processed_temperature_data = preprocess_temporal(temperature_data)

# Display the processed DataFrame to verify changes
print(processed_temperature_data.head())


   Region  Country  State     City  Month  Day    Year  AvgTemperature
0  Africa  Algeria    NaN  Algiers      1    1  1995.0            64.2
1  Africa  Algeria    NaN  Algiers      1    2  1995.0            49.4
2  Africa  Algeria    NaN  Algiers      1    3  1995.0            48.8
3  Africa  Algeria    NaN  Algiers      1    4  1995.0            46.4
4  Africa  Algeria    NaN  Algiers      1    5  1995.0            47.9
Error preprocessing temporal data: "None of [Index(['region', 'country', 'state', 'city', 'year', 'month', 'day',\n       'average_temperature'],\n      dtype='object')] are in the [columns]"
Empty DataFrame
Columns: []
Index: []


In [27]:
import pandas as pd
from transformers import BertTokenizer
from sklearn.model_selection import train_test_split

# Load Text Data
text_data = pd.read_csv('/content/Mental-Health-Twitter.csv')

# Initialize the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Preprocessing Text Data
text_data['input_ids'] = text_data['post_text'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True, truncation=True, padding='max_length', max_length=128))

# Split the text data
X_text_train, X_text_test, y_text_train, y_text_test = train_test_split(text_data['input_ids'].tolist(), text_data['label'], test_size=0.2, random_state=42)


In [28]:
# Check if 'input_ids' exists in the DataFrame
if 'input_ids' not in text_data.columns:
    raise KeyError("'input_ids' column is missing from the text_data DataFrame")


In [29]:
# Add a common column for merging if needed
text_data['common_id'] = text_data.index
image_data['common_id'] = image_data.index
temporal_data['common_id'] = temporal_data.index

# Check columns available for merging
print("Text Data Columns:", text_data.columns)
print("Image Data Columns:", image_data.columns)
print("Temporal Data Columns:", temporal_data.columns)

# Attempt to merge datasets
try:
    combined_data = pd.merge(text_data, image_data, on='common_id', how='inner')
    combined_data = pd.merge(combined_data, temporal_data, on='common_id', how='inner')
except KeyError as e:
    print(f"Merge Error: {e}")
    combined_data = pd.DataFrame()  # Initialize an empty DataFrame in case of error

# Continue processing if merging was successful
if not combined_data.empty:
    print("Datasets merged successfully!")

Text Data Columns: Index(['Unnamed: 0', 'post_id', 'post_created', 'post_text', 'user_id',
       'followers', 'friends', 'favourites', 'statuses', 'retweets', 'label',
       'input_ids', 'common_id'],
      dtype='object')
Image Data Columns: Index(['emotion', 'pixels', 'Usage', 'pixel_size', 'common_id'], dtype='object')
Temporal Data Columns: Index(['Region', 'Country', 'State', 'City', 'Month', 'Day', 'Year',
       'AvgTemperature', 'common_id'],
      dtype='object')
Datasets merged successfully!


In [30]:
# Separate features and labels
X_text = combined_data['input_ids'].tolist()
X_image = np.stack(combined_data['pixels'].values)
X_temporal = combined_data.drop(columns=['label', 'input_ids', 'pixels', 'common_id'])  # Remove non-feature columns
y = combined_data['label']

# Print shapes to verify
print(f"Shape of X_text: {np.array(X_text).shape}")
print(f"Shape of X_image: {np.array(X_image).shape}")
print(f"Shape of X_temporal: {np.array(X_temporal).shape}")
print(f"Shape of y: {np.array(y).shape}")


Shape of X_text: (5123, 128)
Shape of X_image: (5123, 48, 48, 1)
Shape of X_temporal: (5123, 21)
Shape of y: (5123,)


In [31]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_text_train, X_text_test, X_image_train, X_image_test, X_temporal_train, X_temporal_test, y_train, y_test = train_test_split(
    X_text, X_image, X_temporal, y, test_size=0.2, random_state=42
)

# Convert lists to numpy arrays for training
X_text_train = np.array(X_text_train)
X_text_test = np.array(X_text_test)
X_image_train = np.array(X_image_train)
X_image_test = np.array(X_image_test)
X_temporal_train = np.array(X_temporal_train)
X_temporal_test = np.array(X_temporal_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

# Print shapes again to verify
print(f"Shape of X_text_train: {X_text_train.shape}")
print(f"Shape of X_image_train: {X_image_train.shape}")
print(f"Shape of X_temporal_train: {X_temporal_train.shape}")
print(f"Shape of y_train: {y_train.shape}")


Shape of X_text_train: (4098, 128)
Shape of X_image_train: (4098, 48, 48, 1)
Shape of X_temporal_train: (4098, 21)
Shape of y_train: (4098,)


In [35]:
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, Flatten, LSTM, Concatenate
from tensorflow.keras.models import Model

# Define the model
input_text = Input(shape=(128,))  # Adjust shape as needed
input_image = Input(shape=(256, 256, 3))  # Adjust shape as needed

# Check if X_temporal_train is 1D, if so, reshape it
if len(X_temporal_train.shape) == 1:
    X_temporal_train = X_temporal_train.reshape(-1, 1)

input_temporal = Input(shape=(1, X_temporal_train.shape[1]))  # Timesteps=1, Features=X_temporal_train.shape[1]

x_text = Dense(128, activation='relu')(input_text)
x_image = Conv2D(32, (3, 3), activation='relu')(input_image)
x_image = MaxPooling2D((2, 2))(x_image)
x_image = Flatten()(x_image)
x_image = Dense(128, activation='relu')(x_image)

x_temporal = LSTM(64)(input_temporal)
x_temporal = Dense(128, activation='relu')(x_temporal)

combined = Concatenate()([x_text, x_image, x_temporal])
output = Dense(7, activation='softmax')(combined)  # Adjust number of classes if needed

model = Model(inputs=[input_text, input_image, input_temporal], outputs=output)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_10            │ (None, 256, 256, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 254, 254, 32)   │            896 │ input_layer_10[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_1           │ (None, 127, 127, 32)   │              0 │ conv2d_1[0][0]         │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_11            │ (None, 1, 21)          │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_9             │ (None, 128)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten_1 (Flatten)       │ (None, 516128)         │              0 │ max_pooling2d_1[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_1 (LSTM)             │ (None, 64)             │         22,016 │ input_layer_11[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, 128)            │         16,512 │ input_layer_9[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_3 (Dense)           │ (None, 128)            │     66,064,512 │ flatten_1[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_4 (Dense)           │ (None, 128)            │          8,320 │ lstm_1[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate (Concatenate) │ (None, 384)            │              0 │ dense_2[0][0],         │
│                           │                        │                │ dense_3[0][0],         │
│                           │                        │                │ dense_4[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_5 (Dense)           │ (None, 7)              │          2,695 │ concatenate[0][0]      │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 66,114,951 (252.21 MB)

 Trainable params: 66,114,951 (252.21 MB)

 Non-trainable params: 0 (0.00 B)

In [36]:
print(f"Shape of X_text_train: {X_text_train.shape}")
print(f"Shape of X_image_train: {X_image_train.shape}")
print(f"Shape of X_temporal_train: {X_temporal_train.shape}")
print(f"Shape of y_train: {y_train.shape}")

print(f"Shape of X_text_test: {X_text_test.shape}")
print(f"Shape of X_image_test: {X_image_test.shape}")
print(f"Shape of X_temporal_test: {X_temporal_test.shape}")
print(f"Shape of y_test: {y_test.shape}")


Shape of X_text_train: (4098, 128)
Shape of X_image_train: (4098, 48, 48, 1)
Shape of X_temporal_train: (4098, 21)
Shape of y_train: (4098,)
Shape of X_text_test: (1025, 128)
Shape of X_image_test: (1025, 48, 48, 1)
Shape of X_temporal_test: (1025, 21)
Shape of y_test: (1025,)
